<a href="https://colab.research.google.com/github/jithinraj9895/CopyHelper-CH/blob/master/Clean_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
   from torchvision.datasets import ImageFolder
   from torchvision.transforms import ToTensor
   import torchvision.transforms as transforms

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
import torch

import os
dir = "/content/drive/MyDrive/mnist_stego"

dataset = ImageFolder(dir, transform= transforms.Compose([ transforms.Grayscale(num_output_channels=1), ToTensor()]))

train_dl = DataLoader(dataset, batch_size=10,shuffle= True)

In [ ]:
print(dataset.classes)

['cover', 'steg']


In [ ]:
data = next(iter(train_dl))
inputs, labels = data

print(inputs.shape)
print(labels)

torch.Size([10, 1, 28, 28])
tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 0])


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import PIL as image
import pickle as pl

srm_dir = "/content/drive/MyDrive"
SRM_npy = np.load(os.path.join(srm_dir,"SRM_Kernels.npy"))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def activation(x):
  a = torch.ones(x.shape)
  a = torch.mul(a, 3)
  return torch.where(x > -3,x,a)


class the_tlu(nn.Module):
    def __init__(self):
        super(the_tlu, self).__init__()
        
    def forward(self, x):
        return activation(x)

In [ ]:
import torch
import torch.nn as nn
from torch.optim.adamax import Adamax
from torch.optim.adadelta import Adadelta
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import Parameter

In [ ]:


class SRMConv2d(nn.Module):

    def __init__(self, stride=1, padding=0):
        super(SRMConv2d, self).__init__()
        self.in_channels = 1
        self.out_channels = 30
        self.kernel_size = (5, 5)
        if isinstance(stride, int):
            self.stride = (stride, stride)
        else:
            self.stride = stride
        if isinstance(padding, int):
            self.padding = (padding, padding)
        else:
            self.padding = padding
        self.dilation = (1, 1)
        self.transpose = False
        self.output_padding = (0,)
        self.groups = 1
        self.weight = Parameter(torch.Tensor(30, 1, 5, 5), requires_grad=True)
        self.bias = Parameter(torch.Tensor(30), requires_grad=True)
        self.reset_parameters()

    def reset_parameters(self):
        self.weight.data.numpy()[:] = SRM_npy
        self.bias.data.zero_()

    def forward(self, input):
        return F.conv2d(input, self.weight, self.bias, self.stride, self.padding,
                        self.dilation, self.groups)


class BlockA(nn.Module):

    def __init__(self, in_planes, out_planes, norm_layer=None,activation = nn.ReLU(inplace=True)):
        super(BlockA, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(in_planes, out_planes)
        self.bn1 = norm_layer(out_planes)
        self.conv2 = conv3x3(out_planes, out_planes)
        self.bn2 = norm_layer(out_planes)

        self.relu = activation

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

In [ ]:

def accuracy(outputs, labels):
    _, argmax = torch.max(outputs, 1)
    return (labels == argmax.squeeze()).float().mean()


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)



In [ ]:
class BlockB(nn.Module):

    def __init__(self, in_planes, out_planes, norm_layer=None):
        super(BlockB, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(in_planes, out_planes, stride=2)
        self.bn1 = norm_layer(out_planes)
        self.conv2 = conv3x3(out_planes, out_planes)
        self.bn2 = norm_layer(out_planes)
        # self.pool = nn.AvgPool2d(3, stride=2, padding=1)

        self.shortcut_conv = conv1x1(in_planes, out_planes, stride=2)
        self.shortcut_bn = norm_layer(out_planes)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        # out = self.pool(out)

        identity = self.shortcut_conv(identity)
        identity = self.shortcut_bn(identity)

        out += identity
        out = self.relu(out)

        return out


In [ ]:
class KeNet(nn.Module):

    def __init__(self, norm_layer=None, zero_init_residual=True, p=0.5):
        super(KeNet, self).__init__()

        self.zero_init_residual = zero_init_residual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.srm = SRMConv2d(1, 0)
        self.bn1 = norm_layer(30)

        self.A1 = BlockA(30, 30, norm_layer=norm_layer)
        self.A2 = BlockA(30, 30, norm_layer=norm_layer)
        self.AA = BlockA(30, 30, norm_layer=norm_layer)

        # self.B1 = BlockB(30, 30, norm_layer=norm_layer)
        # self.B2 = BlockB(30, 64, norm_layer=norm_layer)

        self.B3 = BlockB(30, 64, norm_layer=norm_layer)
        self.A3 = BlockA(64, 64, norm_layer=norm_layer)

        self.B4 = BlockB(64, 128, norm_layer=norm_layer)
        self.A4 = BlockA(128, 128, norm_layer=norm_layer)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.bnfc = nn.BatchNorm1d(128)
        self.relu = nn.ReLU(inplace=True)
        # self.fcfusion = nn.Linear(128, 128) #4
        self.fc = nn.Linear(128 * 4 + 1, 2)
        self.dropout = nn.Dropout(p=p)

        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # nn.init.xavier_uniform_(m.weight)
                # nn.init.constant_(m.bias, 0.2)
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, std=0.01)

        if self.zero_init_residual:
            for m in self.modules():
                if isinstance(m, (BlockA, BlockB)):
                    nn.init.constant_(m.bn2.weight, 0)

    def extract_feat(self, x):
        x = x.float()
        out = self.srm(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.A1(out)
        out = self.A2(out)
        out = self.AA(out)

        # out = self.B1(out)
        # out = self.B2(out)

        out = self.B3(out)
        out = self.A3(out)

        out = self.B4(out)
        out = self.A4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), out.size(1))

        # out = self.relu(out)
        # out = self.bnfc(out)

        return out

    def forward(self, *args):
        ############# statistics fusion start #############
        feats = torch.stack(
            [self.extract_feat(subarea) for subarea in args], dim=0
        )

        euclidean_distance = F.pairwise_distance(feats[0], feats[1], eps=1e-6,
                                                 keepdim=True)

        if feats.shape[0] == 1:
            final_feat = feats.squeeze(dim=0)
        else:
            # feats_sum = feats.sum(dim=0)
            # feats_sub = feats[0] - feats[1]
            feats_mean = feats.mean(dim=0)
            feats_var = feats.var(dim=0)
            feats_min, _ = feats.min(dim=0)
            feats_max, _ = feats.max(dim=0)

            '''feats_sum = feats.sum(dim=0)
            feats_sub = abs(feats[0] - feats[1])
            feats_prod = feats.prod(dim=0)
            feats_max, _ = feats.max(dim=0)'''
            
            #final_feat = torch.cat(
            #    [feats[0], feats[1], feats[0], feats[1]], dim=-1
            #    #[euclidean_distance, feats_sum, feats_sub, feats_prod, feats_max], dim=-1
            #)

            final_feat = torch.cat(
                [euclidean_distance, feats_mean, feats_var, feats_min, feats_max], dim=-1
                #[euclidean_distance, feats_sum, feats_sub, feats_prod, feats_max], dim=-1
            )

        out = self.dropout(final_feat)
        # out = self.fcfusion(out)
        # out = self.relu(out)
        out = self.fc(out)

        return out, feats[0], feats[1]


In [ ]:
def preprocess_data(images, labels, random_crop):
    # images of shape: NxCxHxW
    if images.ndim == 5:  # 1xNxCxHxW
        images = images.squeeze(0)
        labels = labels.squeeze(0)
    h, w = images.shape[-2:]

    if random_crop:
        ch = random.randint(h * 3 // 4, h)  # h // 2      #256
        cw = random.randint(w * 3 // 4, w)  # square ch   #256

        h0 = random.randint(0, h - ch)  # 128
        w0 = random.randint(0, w - cw)  # 128
    else:
        ch, cw, h0, w0 = h, w, 0, 0
 
        cw = cw & ~1
        inputs = [
            images[..., h0:h0 + ch, w0:w0 + cw // 2],
            images[..., h0:h0 + ch, w0 + cw // 2:w0 + cw]
        ]
        
    return inputs, labels

In [ ]:
class ContrastiveLoss(nn.Module):

    def __init__(self, margin=1.25):  # margin=2
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        label = label.to(torch.float32)
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean(
            (1 - label) * torch.pow(euclidean_distance, 2) +
            label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        )

        return loss_contrastive

In [ ]:
model = KeNet(norm_layer=None, zero_init_residual=True, p=0.5)

In [ ]:
criterion_1 = nn.CrossEntropyLoss()
criterion_2 = ContrastiveLoss(margin=1)

In [ ]:
data = next(iter(train_dl))

In [ ]:
imgs, label = data
print(imgs.shape, label.shape)

torch.Size([10, 1, 28, 28]) torch.Size([10])


In [ ]:
inputs , labels = preprocess_data(*data,False)
output, feat_0, feat_1 = model(*inputs)

print(output,feat_0,feat_1)
print(output.shape)

tensor([[ 0.0618,  0.1070],
        [-0.0443,  0.1546],
        [ 0.0376, -0.0066],
        [-0.0887, -0.0080],
        [ 0.0158,  0.1275],
        [-0.0549,  0.0499],
        [ 0.1117,  0.0413],
        [ 0.0065, -0.0989],
        [ 0.1302,  0.1392],
        [ 0.0423,  0.0331]], grad_fn=<AddmmBackward>) tensor([[0.4430, 0.3448, 0.2581,  ..., 0.5128, 0.2549, 0.2921],
        [0.2272, 0.3317, 0.3770,  ..., 0.3296, 0.2352, 0.1929],
        [0.4223, 0.3265, 0.3652,  ..., 0.2000, 0.3763, 0.1973],
        ...,
        [0.2317, 0.3008, 0.3271,  ..., 0.1344, 0.2285, 0.2054],
        [0.1363, 0.3044, 0.4646,  ..., 0.7615, 0.2172, 0.1407],
        [0.2927, 0.3406, 0.2817,  ..., 0.0429, 0.2523, 0.1837]],
       grad_fn=<SelectBackward>) tensor([[0.2337, 0.5625, 0.2243,  ..., 0.2888, 0.3616, 0.4264],
        [0.3408, 0.2916, 0.5200,  ..., 0.3058, 0.3985, 0.3405],
        [0.2474, 0.3210, 0.3281,  ..., 0.5844, 0.2064, 0.1903],
        ...,
        [0.3382, 0.4267, 0.1904,  ..., 0.3474, 0.2619, 0.3

In [ ]:
print(feat_0.shape)

torch.Size([10, 128])


In [ ]:
optimizer = Adamax(model.parameters(), lr=0.001, eps=1e-08, weight_decay=0.0001)

In [ ]:
optimizer.zero_grad()

In [ ]:
loss = criterion_1(output, labels) + 0.1 * criterion_2(feat_0, feat_1, labels)

In [ ]:
def train(epoch):
  running_loss, running_accuracy = 0., 0.
  for batch in range(epoch):
    data = next(iter(train_dl))
    inputs , labels = preprocess_data(*data,False)
    optimizer.zero_grad()
    output, feat_0, feat_1 = model(*inputs)
    loss = criterion_1(output, labels) + 0.1 * criterion_2(feat_0, feat_1, labels)

    the_accuracy = accuracy(output, labels).item()
    running_accuracy += the_accuracy
    running_loss += loss.item()

    print('EPOCH :',batch,"RUNNING ACCURACY :",running_accuracy, '  ',"RUNNING LOSS:", running_loss, '\n')
    loss.backward()
    optimizer.step()

In [ ]:
train(50)

EPOCH : 0 RUNNING ACCURACY : 0.30000001192092896    RUNNING LOSS: 1.181064248085022 

EPOCH : 1 RUNNING ACCURACY : 0.6000000238418579    RUNNING LOSS: 2.239732265472412 

EPOCH : 2 RUNNING ACCURACY : 0.9000000357627869    RUNNING LOSS: 3.277995467185974 

EPOCH : 3 RUNNING ACCURACY : 1.3000000417232513    RUNNING LOSS: 4.325651168823242 

EPOCH : 4 RUNNING ACCURACY : 2.0000000298023224    RUNNING LOSS: 5.093396067619324 

EPOCH : 5 RUNNING ACCURACY : 2.400000035762787    RUNNING LOSS: 6.2436524629592896 

EPOCH : 6 RUNNING ACCURACY : 2.700000047683716    RUNNING LOSS: 7.316940665245056 

EPOCH : 7 RUNNING ACCURACY : 3.1000000536441803    RUNNING LOSS: 8.397708296775818 

EPOCH : 8 RUNNING ACCURACY : 3.4000000655651093    RUNNING LOSS: 9.480631470680237 

EPOCH : 9 RUNNING ACCURACY : 4.000000089406967    RUNNING LOSS: 10.254092335700989 

EPOCH : 10 RUNNING ACCURACY : 4.100000090897083    RUNNING LOSS: 11.5436429977417 

EPOCH : 11 RUNNING ACCURACY : 4.600000090897083    RUNNING LOSS: 1

In [ ]:
test = next(iter(train_dl))
inputs , labels = preprocess_data(*test,False)
output, feat0, feat1 = model(*inputs)
print(output)
print(labels)

tensor([[-0.2144,  0.1171],
        [-0.1451,  0.1237],
        [-0.2418,  0.0516],
        [ 0.0081,  0.1123],
        [-0.2175,  0.1187],
        [-0.1294,  0.0470],
        [-0.0567, -0.0321],
        [-0.1172,  0.1819],
        [-0.1181,  0.0820],
        [-0.1862,  0.0399]], grad_fn=<AddmmBackward>)
tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0])


In [ ]:
F.softmax(output,dim = 0)

tensor([[0.0927, 0.1032],
        [0.0994, 0.1039],
        [0.0902, 0.0966],
        [0.1159, 0.1027],
        [0.0925, 0.1033],
        [0.1010, 0.0962],
        [0.1086, 0.0889],
        [0.1022, 0.1101],
        [0.1021, 0.0996],
        [0.0954, 0.0955]], grad_fn=<SoftmaxBackward>)